In [1]:
!pip install --quiet git+https://github.com/CSIRO-enviro-informatics/loci-scripts.git@master
!pip install --quiet numpy pandas
!pip install --quiet seaborn

# Visualise LGA Data - Example DHHS Vic Covid-19

This notebook show how to visualise tabular data location coded to LGA on a map using Loc-I APIs.

In [2]:
import numpy as np
import pandas as pd
df = pd.read_csv('vic_covid_20200723.csv', delimiter = ',')
#show the csv file read in
df

,LGA,Active cases (current),LGA_ID,LGA_URI
0,Alpine,1,20110,http://linked.data.gov.au/dataset/asgs2016/loc...
1,Ararat,5,20260,http://linked.data.gov.au/dataset/asgs2016/loc...
2,Ballarat,23,20570,http://linked.data.gov.au/dataset/asgs2016/loc...
3,Banyule,265,20660,http://linked.data.gov.au/dataset/asgs2016/loc...
4,Bass Coast,6,20740,http://linked.data.gov.au/dataset/asgs2016/loc...
...,...,...,...,...
74,Wodonga,1,27170,http://linked.data.gov.au/dataset/asgs2016/loc...
75,Wyndham,694,27260,http://linked.data.gov.au/dataset/asgs2016/loc...
76,Yarra,205,27350,http://linked.data.gov.au/dataset/asgs2016/loc...
77,Yarra Ranges,46,27450,http://linked.data.gov.au/dataset/asgs2016/loc...


### Visualise LGA data

In [3]:
#visualise LGA 
import json
import requests
def get_geom(loci_uri):
    payload = {
        "uri": loci_uri
    }
    url = "https://api.loci.cat/api/v1/location/geometry"
    r = requests.get(url, params=payload)
    res = r.json()
    #get the first geom result
    geojson_data = []
    return_code = [-1, '']
    if len( res['geometry']) > 0:
        geojson_data = res['geometry'][0]
        return_code = [0, '']
    if 'meta' in res and 'geom_response_errors' in res['meta']:
        err_message = res['meta']['geom_response_errors']
        return_code = [1, err_message]        
    return (return_code, geojson_data)

In [4]:
import seaborn as sns
from ipywidgets import Text, HTML
from ipyleaflet import (WidgetControl)
import math
target_col = "Active cases (current)"
palette = sns.light_palette("navy", 6, reverse=False)
hexpal = palette.as_hex()
#df['quantiles'] = pd.qcut(df[target_col], q=5,  duplicates="drop")
df['categories'] = pd.cut(df[target_col], [0,1,10,100,200,300,500])
df['hex'] = pd.cut(df[target_col], [0,1,10,100,200,300,500],labels=hexpal)

In [5]:
geom_idx = {}

In [6]:
import ipyleaflet as ipy 
import ipywidgets as ipyw
from ipyleaflet import GeoJSON, Map, Marker, WidgetControl
from ipywidgets import Text, HTML


x_coord = -37.8136
y_coord = 144.9631

map1 = ipy.Map(center=[x_coord, y_coord], zoom=9)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

html = HTML('''Hover over a region''')
html.layout.margin = '0px 20px 20px 20px'
control = WidgetControl(widget=html, position='topright')
map1.add_control(control)

def update_html(feature, **kwargs):
     html.value = '''
     <h4>LGA code: {}</h4>
     <h4>LGA name: {}</h4>
     <h4>No. Cases: {}</h4>
      '''.format(feature['id'], feature['label'], math.floor(feature['num_cases']))

res_df_cols = {
    "LGA" : [],
    "SA2" : [],
    "percent_overlap": [],
    target_col: [],
    "reapportioned_data": [],
}
for index, row in df.iterrows():
    fromFeature = row['LGA_URI']
    lgaName = row['LGA']
    if fromFeature == None or fromFeature == '':
        continue
    value  = row[target_col]
    thegeom = {}
    if fromFeature in geom_idx:
        thegeom = geom_idx[fromFeature]
    else:
        (return_code, geojson_data) = get_geom(fromFeature)
        if(return_code[0] != 0):
            print("Error in fetching {}. Error message: {}".format(fromFeature, return_code[1]))
            continue
        else:
            geom_idx[fromFeature] = geojson_data
            thegeom = geojson_data        
    thegeom['num_cases'] = value
    thegeom['label'] = lgaName
    color = row['hex']
    if row['hex'] == None or row['hex'] == 'NaN' or str(row['hex']) == 'nan':
        color = "#FFFFFF"
    if value > 0:
        curr_geojson_layer = GeoJSON(data=thegeom, 
                    style={
                          'color': 'black', 
                          'opacity': 1, 
                          'weight':1, 
                          'fillColor': color, 
                          'fillOpacity': 0.7},
                     hover_style={
                          'color': 'black', 
                          'opacity': 1, 
                          'weight':1, 
                          'fillColor': color, 
                          'fillOpacity': 0.9,                    
                    })
        curr_geojson_layer.on_hover(update_html)
        map1.add_layer(curr_geojson_layer)

    

In [7]:
map1

Map(center=[-37.8136, 144.9631], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…